In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy import sparse
import pickle

# Load the dataset
data = pd.read_csv(r'P:\\project ides for final ptoject\\project\\IDS_Federated learning model\\model\\cleaned_x_iiotid_dataset.csv', low_memory=False)

# Specify the columns to keep
columns_to_keep = ['Scr_port', 'Des_port', 'Protocol', 'Service', 'Scr_bytes', 'Des_bytes', 'Conn_state', 'anomaly_alert', 'class3']
data_filtered = data[columns_to_keep]

# Define features and labels
X = data_filtered.drop(columns=['class3'])
y = data_filtered['class3']

# Handle categorical and numerical columns
categorical_columns = ['Protocol', 'Service', 'Conn_state']
numerical_columns = X.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Preprocessing pipeline
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numerical_columns),
    ('cat', OneHotEncoder(sparse_output=False), categorical_columns)
])

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Preprocess the data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Convert to dense format if sparse
X_train = X_train.toarray() if sparse.issparse(X_train) else X_train
X_test = X_test.toarray() if sparse.issparse(X_test) else X_test

# Convert labels to numerical format
y_train = pd.factorize(y_train)[0]
y_test = pd.factorize(y_test)[0]

# One-hot encode if multi-class classification
num_classes = len(np.unique(y_train))
if num_classes > 2:
    y_train = np.eye(num_classes)[y_train]
    y_test = np.eye(num_classes)[y_test]

# Activation functions
def leaky_relu(Z, alpha=0.01):
    return np.where(Z > 0, Z, alpha * Z)

def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

def softmax(Z):
    exp_Z = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return exp_Z / np.sum(exp_Z, axis=1, keepdims=True)

# Initialize weights
def initialize_weights(layers):
    np.random.seed(42)
    weights = []
    biases = []
    for i in range(1, len(layers)):
        weight = np.random.randn(layers[i-1], layers[i]) * np.sqrt(2 / layers[i-1])
        bias = np.zeros((1, layers[i]))
        weights.append(weight)
        biases.append(bias)
    return weights, biases  # Return as lists, not NumPy arrays

# Forward propagation
def forward_propagation(X, weights, biases):
    A = X
    caches = []
    for i in range(len(weights) - 1):
        Z = np.dot(A, weights[i]) + biases[i]
        A = leaky_relu(Z)
        caches.append((A, Z))
    Z_final = np.dot(A, weights[-1]) + biases[-1]
    A_final = softmax(Z_final) if num_classes > 2 else sigmoid(Z_final)
    caches.append((A_final, Z_final))
    return A_final, caches

# Backward propagation
def backward_propagation(X_batch, y_true, caches, weights, biases, learning_rate):
    m = X_batch.shape[0]
    A_final, Z_final = caches[-1]
    y_true = np.eye(num_classes)[y_true] if len(y_true.shape) == 1 else y_true

    dZ_final = A_final - y_true
    dW_final = np.dot(caches[-2][0].T, dZ_final) / m
    db_final = np.sum(dZ_final, axis=0, keepdims=True) / m
    weights[-1] -= learning_rate * dW_final
    biases[-1] -= learning_rate * db_final

    dA = dZ_final
    for i in reversed(range(len(weights) - 1)):
        A_prev, Z_prev = caches[i]
        dZ = dA.dot(weights[i+1].T) * (Z_prev > 0)
        dW = np.dot(caches[i-1][0].T, dZ) / m if i > 0 else np.dot(X_batch.T, dZ) / m
        db = np.sum(dZ, axis=0, keepdims=True) / m
        weights[i] -= learning_rate * dW
        biases[i] -= learning_rate * db
        dA = dZ

    return weights, biases

# Training the model
def train(X_train, y_train, layers, epochs=30, batch_size=64, learning_rate=0.001):
    weights, biases = initialize_weights(layers)
    m = X_train.shape[0]
    for epoch in range(epochs):
        for i in range(0, m, batch_size):
            X_batch = X_train[i:i + batch_size]
            y_batch = y_train[i:i + batch_size]
            y_batch = np.eye(num_classes)[y_batch] if len(y_batch.shape) == 1 else y_batch
            y_pred, caches = forward_propagation(X_batch, weights, biases)
            weights, biases = backward_propagation(X_batch, y_batch, caches, weights, biases, learning_rate)

        if epoch % 5 == 0:
            print(f"Epoch {epoch+1}/{epochs}")

    return weights, biases

# Save weights and biases to a pickle file
def save_weights_to_pickle(weights, biases, filename):
    # Store the weights and biases as numpy arrays inside a dictionary
    data = {
        "weights": [np.array(w) for w in weights],  # Convert list of arrays to numpy arrays
        "biases": [np.array(b) for b in biases],    # Convert list of arrays to numpy arrays
    }
    
    # Save the dictionary to a pickle file
    with open(filename, 'wb') as file:
        pickle.dump(data, file)
    print(f"Weights and biases saved to {filename}")

# Load weights and biases from a pickle file
def load_weights_from_pickle(filename):
    with open(filename, 'rb') as file:
        data = pickle.load(file)
    weights = data["weights"]
    biases = data["biases"]
    return weights, biases

# Define layers and train the model
input_size = X_train.shape[1]
layers = [input_size, 512, 256, 128, num_classes]
weights, biases = train(X_train, y_train, layers, epochs=30, batch_size=64, learning_rate=0.001)

# Save weights and biases
save_weights_to_pickle(weights, biases, "model_weights_and_biases.pickle")

# Load weights and biases (example usage)
loaded_weights, loaded_biases = load_weights_from_pickle("model_weights_and_biases.pickle")

# Evaluate the model on the test set
y_pred, _ = forward_propagation(X_test, loaded_weights, loaded_biases)

# Convert predictions back to labels
y_pred_labels = np.argmax(y_pred, axis=1) if num_classes > 2 else (y_pred > 0.5).astype(int)

# Calculate and print metrics
accuracy = accuracy_score(y_test, y_pred_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print confusion matrix
cm = confusion_matrix(y_test, y_pred_labels)
print("Confusion Matrix:")
print(cm)
